# Video Transmission Distortion Simulation in Colab

In [ ]:
# Install necessary packages for ns-3 and pyshark
!apt-get install -y ns3
!pip install pyshark
!pip install opencv-python-headless
    

In [ ]:
import ns.core
import ns.network
import ns.point_to_point
import ns.internet
import ns.applications
import numpy as np
import cv2
from IPython.display import Video
from IPython.core.display import display

# Initialize the network components
nodes = ns.network.NodeContainer()
nodes.Create(2)
point_to_point = ns.point_to_point.PointToPointHelper()
point_to_point.SetDeviceAttribute("DataRate", ns.core.StringValue("5Mbps"))
point_to_point.SetChannelAttribute("Delay", ns.core.StringValue("20ms"))

devices = point_to_point.Install(nodes)
rate_error_model = ns.network.RateErrorModel()
rate_error_model.SetAttribute("ErrorRate", ns.core.DoubleValue(0.0001))

for device in devices:
    device.SetReceiveErrorModel(rate_error_model)

stack = ns.internet.InternetStackHelper()
stack.Install(nodes)

address = ns.internet.Ipv4AddressHelper()
address.SetBase(ns.network.Ipv4Address("10.1.1.0"), ns.network.Ipv4Mask("255.255.255.0"))
interfaces = address.Assign(devices)

# Video transmission simulation setup
packet_sink_helper = ns.applications.PacketSinkHelper("ns3::TcpSocketFactory",
                                                       ns.network.InetSocketAddress(ns.network.Ipv4Address.GetAny(), 8080))
sink_apps = packet_sink_helper.Install(nodes.Get(1))
sink_apps.Start(ns.core.Seconds(1.0))
sink_apps.Stop(ns.core.Seconds(10.0))

on_off_helper = ns.applications.OnOffHelper("ns3::TcpSocketFactory",
                                            ns.network.InetSocketAddress(interfaces.GetAddress(1), 8080))
on_off_helper.SetAttribute("PacketSize", ns.core.UintegerValue(512))
on_off_helper.SetAttribute("DataRate", ns.core.StringValue("5Mbps"))
source_apps = on_off_helper.Install(nodes.Get(0))
source_apps.Start(ns.core.Seconds(1.5))
source_apps.Stop(ns.core.Seconds(9.5))

# Load and process the input video
input_video_path = 'input_video.mp4'  # Ensure you have this video in the working directory
output_video_path = 'processed_video.mp4'

cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
codec = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, codec, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Simulate distortion by flipping the frame horizontally
    frame = cv2.flip(frame, 1)
    out.write(frame)

cap.release()
out.release()

# Run the network simulation
ns.core.Simulator.Run()
ns.core.Simulator.Destroy()

# Display the processed video
display(Video(output_video_path))
    